In [ ]:
from os import listdir
from os.path import join, isfile
from lxml import etree
from src.data import PackageManager
from src.data import XbrlConst
from collections import defaultdict
from src.data import xbrl2rdf

In [ ]:
XBRL_TAXONOMY_PATH = join('..', 'data', 'external', 'taxonomies')

taxonomies = [f for f in listdir(XBRL_TAXONOMY_PATH) if isfile(join(XBRL_TAXONOMY_PATH, f)) and f[-3:]=='zip']

manager = PackageManager.Taxonomies(XBRL_TAXONOMY_PATH)

for taxonomy in taxonomies:
    manager.addPackage(join(XBRL_TAXONOMY_PATH, taxonomy))
manager.rebuildRemappings()
manager.save()
for item in manager.config['packages']:
    print(item['URL'])

In [ ]:
import src.data

In [ ]:
f = src.data.openFileSource(manager.config['packages'][2]['URL'])
f.mappedPaths = manager.config['packages'][1]["remappings"]

In [ ]:
xmlfiles = [file for file in f.dir if (file[-3:]=='xml') and ('eiopa.europa.eu/eu/xbrl' in file)]

In [ ]:
xmlfiles[101]

In [ ]:
roots = list()

for file in xmlfiles[0:50]:
    fp = f.fs.open(file, "r")
    content = fp.read()
    roots.append(etree.fromstring(content, parser=etree.XMLParser(remove_comments=True)))

In [ ]:
for idx, root in enumerate(roots):
    xbrl2rdf.processLinkBase(root, xmlfiles[idx], root.nsmap)

In [ ]:
# referencedNamespaces = set()
# hrefObjects = []
# baseSets = defaultdict(list)
# resources = dict()

# def discoverHref(element):
#     href = element.get("{http://www.w3.org/1999/xlink}href")
#     if href:
#         url, identifier = src.data.UrlUtil.splitDecodeFragment(href)
#         href = (element, url, identifier if len(identifier) > 0 else None)
#         hrefObjects.append(href)
#         return href
#     else:
#         return None

# element_dict = dict()
    
# for idx, root in enumerate(roots):
#     for linkbaseElement in root.iter(tag="{http://www.xbrl.org/2003/linkbase}linkbase"):
#         for element in linkbaseElement:
#             d = dict()
#             linkQn = etree.QName(element)
#             linkrole = element.get("{http://www.w3.org/1999/xlink}role", "")
#             if linkQn.namespace == XbrlConst.link:
#                 if linkQn.localname in ["roleRef", "arcroleRef"]:
#                     href = discoverHref(element)
#                     if href is None:
#                         print("error")
#             if element.get("{http://www.w3.org/1999/xlink}type") == "extended":
#                 d_links = dict()
#                 arcrolesFound = set()
#                 for linkElement in element.iterchildren():
#                     arcrolesFound = set()
#                     xlinkType = linkElement.get("{http://www.w3.org/1999/xlink}type")
#                     modelResource = None
#                     if xlinkType == "locator":
#                         href = discoverHref(linkElement)
#                         modelResource = linkElement
#                         d['locator'] = linkElement
#                         d['href'] = href
#                     elif xlinkType == "arc":
#                         arcQn = etree.QName(linkElement)
#                         arcrole = linkElement.get("{http://www.w3.org/1999/xlink}arcrole")
#                         if arcrole not in arcrolesFound:
#                             baseSetKeys = [(arcrole, linkrole, linkQn, arcQn)]
#                             baseSetKeys.append((arcrole, linkrole, None, None))
#                             baseSetKeys.append((arcrole, None, None, None))
#                             if XbrlConst.isDimensionArcrole(arcrole):
#                                 d['arc'] = 'XBRL-dimensions'
#                                 baseSetKeys.append(("XBRL-dimensions", linkrole, None, None))
#                                 baseSetKeys.append(("XBRL-dimensions", None, None, None))
#                             if XbrlConst.isFormulaArcrole(arcrole):
#                                 d['arc'] = 'XBRL-formulae'
#                                 baseSetKeys.append(("XBRL-formulae", linkrole, None, None))
#                                 baseSetKeys.append(("XBRL-formulae", None, None, None))
#                             if XbrlConst.isTableRenderingArcrole(arcrole):
#                                 d['arc'] = 'Table-rendering'
#                                 baseSetKeys.append(("Table-rendering", linkrole, None, None)) 
#                                 baseSetKeys.append(("Table-rendering", None, None, None)) 
#                             for baseSetKey in baseSetKeys:
#                                 baseSets[baseSetKey].append(element)
#                             arcrolesFound.add(arcrole)
#                     elif xlinkType == "resource": 
#                         d['resource'] = linkElement
#                         modelResource = linkElement
#                     if modelResource is not None:
#                         if element not in resources.keys():
#                             resources[element] = defaultdict(list)
#                         resources[element][linkElement.get("{http://www.w3.org/1999/xlink}label")].append(modelResource)
#                         d_links[linkElement.get("{http://www.w3.org/1999/xlink}label")] = modelResource
#                 d['links'] = d_links
#                 element_dict[element] = d
# #             else:
# #                 print("Not extended: " + str(element.get("{http://www.w3.org/1999/xlink}type")))

In [ ]:
list(element_dict.values())[411:412]

In [ ]:
a = list(resources.values())[420]
a['label_s2c_x0'][0].text